In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
!pip install lapjv
from lapjv import lapjv
# Any results you write to the current directory are saved as output.

# Introduction


At this point of the competition I think most of us have read [this great solution](https://www.kaggle.com/martinpiotte/whale-recognition-model-with-score-0-78563) by** @martinpiotte**. (If you haven't, you should). I've also ported it [here](https://www.kaggle.com/suicaokhoailang/martin-piotte-s-siamese-baseline), and it seems like using Martin's old model can achieve 0.822 LB as reported by [this kernel](https://www.kaggle.com/seesee/siamese-pretrained-0-822).

I think the most interesting part of the solution is the traing data generator. As it was a siamese network, for each training step we have to chose a pair of images for comparison. The selection strategy, according to the authors:

- Exactly 50% of the pairs are for matching whales, and 50% for different whales;
- Each image from the training set is used exactly 4 times per epoch: A and B images of matching whales, A and B images of different whale pairs;
- Pairs of images of different whales are selected to be difficult for the network to distinguish at a given stage of the training. This is inspired from adversarial training: find pairs of images that are from different whales, but that are still very similar from the model perspective.

Choosing the matching pairs are easy, but how do we select the unmatched ones to maximize the model's robustness and prevent overfitting? That's when the linear assignment problem (LAP) comes into play.

# Linear Assigment Problem

An example of LAP that we are given n facilities and n locations and a $nxn$ matrix $C$ where $c_{ij}$ represents the cost of placing facility $i$ to location $j$. $C$ is the cost matrix, and the objective is to find an optimal assignment of facilities to locations such that the total placement cost is minimized.


A naive algorithm will have an factorial time complexity while Hungarian algorithm can achieve $O(n^3)$. Here, we use the Jonker-Volgenant algorithm, also $O(n^3)$ but faster than the former in practice.

If we treat and the facilities and the locations as the whales, and the cost is the *dissimilarity* between them, **our problem become matching one whale to exactly one another so that the total dissimilarity of all the matches is minimized** (and the similarity is maximized) i.e we try to pick out a subset of our training data so that it *confuses* the model the most, becaus**e each image in each pair belong to a different whale, *yet* they look very similar according to the model's current parameters.**


# How It Applies


There are a few tweaks. The cost matrix in our case is symmetrical and it looks like below. Here I assume 0 means totally similar and 1 for totally different. Matched pairs (the main diagonal) will have a score of 0. The scores may be the L2 distance of each image's output from a feature extractor model, or in case of Martin's solution it was a probability of 2 images belong to the same whale. 


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
cost_mat = np.array([ 
   [0, 0.2, 0.3,0.1, 0.15], 
   [0, 0, 0.35,0.12, 0.37], 
   [0, 0, 0, 0.1, 0.15], 
   [0, 0, 0, 0, 0.08], 
   [0, 0, 0, 0, 0], 
 ]) 

cost_mat += cost_mat.T


def draw_cost_mat(cost_mat, subplot=111):
    plt.subplot(subplot)
    size = cost_mat.shape[0]
    x_start = 3.0 
    x_end = 9.0 
    y_start = 6.0 
    y_end = 12.0 
    extent = [x_start, x_end, y_start, y_end]
    jump_x = (x_end - x_start) / (2.0 * size) 
    jump_y = (y_end - y_start) / (2.0 * size) 
    x_positions = np.linspace(start=x_start, stop=x_end, num=size, endpoint=False) 
    y_positions = np.linspace(start=y_start, stop=y_end, num=size, endpoint=False)  

    plt.imshow(cost_mat, extent=extent)
    plt.imshow(cost_mat, extent=extent)         
    for y_index, y in enumerate(y_positions): 
         for x_index, x in enumerate(x_positions): 
            label = np.flip(cost_mat,1)[x_index, y_index] 
            text_x = x + jump_x 
            text_y = y + jump_y 
            plt.text(text_x, text_y, label, color='white', ha='center', va='center')

matplotlib.rcParams['figure.figsize'] = [15, 5]

draw_cost_mat(cost_mat,111)

plt.show() 

Since we only care about finding unmatched pairs, we set the main diagonal's values to ∞ so that the algorithm won't ever pick them up.

For each epoch, lapjv will output a solution consists of n pairs of unmatched whales, when we're done with those pairs we set their similarity to ∞ and running the solver again for another epoch.

Eventually, the cost matrix will run out of finite values after a few epochs and we will need to reset it by running all training images through the feature extractor with the model's updated parameters.

In [ ]:
# set the main diagonal to infinity
cost_mat[np.arange(cost_mat.shape[0]), np.arange(cost_mat.shape[0])] = np.inf

draw_cost_mat(cost_mat, 121)

solved = cost_mat.copy()

# Solving LAP
x = lapjv(solved)[0]
y = np.arange(len(x), dtype=np.int32)
# Set the selected pairs scores to infinity
#  the x,y pair will be used for training
solved[x,y] = np.inf
solved[y,x] = np.inf

draw_cost_mat(solved, 122)

plt.show()

# A Potential Bottleneck

As we keep training our model will get better and better at distinguishing pairs which were previously difficult. That will make all the costs in the cost matrix very close to 1 and to each other. As lapjv becomes horribly time consuming, I think it'll be better to switch to random matching at that point.